In [52]:
import json
import requests
import time

## Load dataset

In [53]:
with open("train-v2.0.json", 'r') as f:
    data= json.load(f)

In [54]:
def get_qa(topic, data):
    q = []
    a = []
    for d in data['data']:
        if d['title']==topic:
            for paragraph in d['paragraphs']:
                for qa in paragraph['qas']:
                    if not qa['is_impossible']:
                        q.append(qa['question'])
                        a.append(qa['answers'][0]['text'])
            return q,a

questions, answers = get_qa(topic='Premier_League', data=data)
print("Number of available questions: {}".format(len(questions)))

Number of available questions: 357


In [55]:
%%bash
docker build . -t qamodel
docker run -d -p 8000:8000 qamodel

#1 [internal] load build definition from Dockerfile
#1 sha256:9c1eedacbf64ad3e7da837b375465fdd9d291899262cc0a88630e898a7d4f9cd
#1 transferring dockerfile: 74B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:3aa65305f3ad52e7e3b6325b535f9092352ce9a40225c7a1bc665b10afd5e24a
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.9.12-slim-buster
#3 sha256:9889230219fc0c4a53bc7a99f30fc55c5e00ba45ee93f775f14f3ee4e8db4d83
#3 DONE 1.9s

#4 [1/5] FROM docker.io/library/python:3.9.12-slim-buster@sha256:830e161433edfe047a23ebc99c12ee0eb1dc0a50e6b5f1c98e869ac271786632
#4 sha256:4b98a2df8eba665dae81f3c30f2c2630879fa964ffc329066b592b88c5dedf39
#4 DONE 0.0s

#6 [internal] load build context
#6 sha256:9b4d4ffb13d8f4fcf6e5d6d46b72dc40b62391a95f58dccfbb911b3c5981bdb3
#6 transferring context: 9.80kB done
#6 DONE 0.0s

#7 [3/5] COPY download_model.sh .
#7 sha256:3fa85e5215dbec421bde4999546f268b8229723518bd9a6b232913d7a79be128
#7 CACHED

#5 [

e280ca9e34b02b92c9ecf642ce1bf9eb2ad87302f501a1314e2354c1265ebcf0


## Test App

In [56]:
time.sleep(10)

In [57]:
import requests

json_data = {
  'questions':questions,
  'answers':answers,
}

response = requests.post(
  'http://0.0.0.0:8000/set_context',
  json=json_data,
)

response.json()

{'message': 'Search context here'}

In [58]:
new_questions = [
    'How many teams compete in the Premier League ?',
    'When does the Premier League starts and finishes ?',
    'Who has the highest number of goals in the Premier League ?',
]

json_data = {
  'questions':new_questions,
}

response = requests.post(
  'http://0.0.0.0:8000/get_answer',
  json=json_data
)

for d in response.json():
  print('\n'.join(["{} : {}".format(k, v) for k,v in d.items()])+'\n')

orig_q : How many teams compete in the Premier League ?
best_q : How many clubs are currently in the Premier League?
best_a : 20

orig_q : When does the Premier League starts and finishes ?
best_q : When does the Premier League have its playing season?
best_a : During the course of a season (from August to May)

orig_q : Who has the highest number of goals in the Premier League ?
best_q : Who has the record for most goals in the Premier League?
best_a : Newcastle United striker Alan Shearer holds the record for most Premier League goals with 260

